In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import json
file = open('./../r_addresses.json')
wallet = json.load(file)

In [9]:
# lock p2tr script
import struct
from key import ECPubKey,ECKey
from time import time
from messages import CTxIn,COutPoint,CTxOut,CTransaction,CTxWitness,CTxInWitness,ser_uint256
from script import tagged_hash, sha256,taproot_construct,SegwitV0SignatureHash, CScript, OP_2,OP_1, OP_CHECKMULTISIG, OP_0,OP_CHECKSIG, OP_DROP, OP_CHECKLOCKTIMEVERIFY, OP_CHECKSEQUENCEVERIFY
import sys
sys.path.append("..")
from ecc import PrivateKey

prev_tx_hx = int("2063c950fa67e26f6afe9035b0078fcc1284088c4cac882ed87f0f8f7d35c2ed", 16)
prv_amount=int(10*100000000)
prv_pub_key_hash = "7d7143a28a3b836e3043f3ba5dbcdc4ee29b7457"
prv_script=bytes.fromhex(f"76a914{prv_pub_key_hash}88ac")
prev_index = 1

p=ECKey()
p.set(0x00db5cdc1663eea25406126e642f72ef7c78f68d249589d2873c613e61929eea)

target_public_key_bytes=bytes.fromhex(wallet[12]["public key"])
target_amount = int(9.999*100000000)

sequence = (1 << 22 ) | (1 << 0)
tx_in =CTxIn(COutPoint(prev_tx_hx, prev_index), nSequence=0xffffffff)

target_public_key=ECPubKey().set(target_public_key_bytes)
mast_scripts = [ CScript([sequence.to_bytes(3,"little"), OP_CHECKSEQUENCEVERIFY, OP_DROP, target_public_key.get_bytes(), OP_CHECKSIG]) ]

locking_script, tweak, control_map = taproot_construct(target_public_key, mast_scripts)

tx_outs = CTxOut(target_amount, locking_script)

tx = CTransaction()
tx.nVersion=2
tx.vin=[tx_in]
tx.vout=[tx_outs]
# tx.nLockTime = round(time() + 60)
tx.calc_sha256()
tx_sig_hash = SegwitV0SignatureHash(prv_script, tx, 0, 0x01, prv_amount)
tx_input_0_wit = CTxInWitness([p.sign_ecdsa(tx_sig_hash) + (1).to_bytes(1, 'big'), p.get_pubkey().get_bytes(False)])
tx.wit.vtxinwit = [tx_input_0_wit]

print(tx.serialize_with_witness().hex())

# Use for Next Transaction
tx1_outs = tx.vout
t = tweak
cmp = control_map

0200000000010159ac4bc319458d7d58fba97b4e9e54c59705610ec18cf69dd6bccabd1428ef720100000000ffffffff016043993b0000000022512052b4f4d6a66f5b763b4c903aa0847b8fb2ec0d1ae8b458d057020daa5ea58ebb024730440220769e0dcd9fc50b71a92e996f018d274a1aa5a1d4e22df3113ff906207825175302200250ba71c5685b3a5175dae6d5d1d5508b0eebfd5aa96618c247b95952caf014012103eb477934368b96afb4e7878b96f29bc4aec8238189652f17a7b33eb0ed6acbee00000000


In [11]:
# unlocking p2tr script path htlc
from messages import ser_string
from script import get_p2wpkh_script, TaprootSignatureHash
from segwit_addr import decode_segwit_address
from key import SECP256K1_ORDER, SECP256K1, SECP256K1_G, int_or_bytes


taproot_locking_tx_id = int("86dec6299835e653d8f62f8bbae21f0b23afe91b332a85195dec7fda0c01d507", 16)
taproot_locking_tx_index = 0

taproot_unlocking_sec_key = ECKey().set(bytes.fromhex("13e4f9204068a232f23fe6305bb6c1fcb6d2c6e53347956a7fbaf2953737ab4c"))

target_amount = int(9.998*100000000)
target_b32_address = wallet[0]["address"]

sequence = (1 << 22) | (1 << 2)

tx_in_0 = CTxIn(COutPoint(taproot_locking_tx_id, taproot_locking_tx_index), nSequence=sequence)
tx_ins = [tx_in_0]


locking_public_key_hash = bytes(decode_segwit_address("bcrt", target_b32_address)[1])
locking_script = get_p2wpkh_script(locking_public_key_hash)
tx_out_0 = CTxOut(target_amount, locking_script)
tx_outs = [tx_out_0]

tx = CTransaction()
tx.nVersion = 2
tx.vin = tx_ins
tx.vout = tx_outs
tx.calc_sha256()

prev_tx_outs = tx1_outs
tx_sig_hash_0 = TaprootSignatureHash(tx, prev_tx_outs, 0X00, 0, True, list(cmp)[0])
taproot_unlocking_sec_key.negate()
tx_sign_0 = taproot_unlocking_sec_key.sign_schnorr(tx_sig_hash_0)

tx_input_0_wit = CTxInWitness([ tx_sign_0, list(cmp)[0], cmp[list(cmp)[0]]])
tx.wit.vtxinwit = [tx_input_0_wit]
print(tx.serialize_with_witness().hex())



0200000000010107d5010cda7fec5d19852a331be9af230b1fe2ba8b2ff6d853e6359829c6de8600000000000400400001bfbc973b000000001600147d7143a28a3b836e3043f3ba5dbcdc4ee29b74570340b16dc3312f55ed067546ee5723ffedc8e71ff1085ab92139bdb1319161ce55c9ce998519bfe940c4d9a8b9e29a66b3df0c17b32870606f019646987062ff39d12803010040b27520728f0593227b1ba7d31914bf2f83d7335e12fdb92c7b39d12999594b61997a19ac21c1728f0593227b1ba7d31914bf2f83d7335e12fdb92c7b39d12999594b61997a1900000000
